cubicasaのHigh quality architecturalの画像を前処理

オリジナル画像のオブジェクトがない箇所を白色化

In [ ]:
!pip install svg.path

In [ ]:
!curl -L -o cubicasa5k.zip\
  https://www.kaggle.com/api/v1/datasets/download/qmarva/cubicasa5k

In [ ]:
!unzip -q cubicasa5k.zip

In [ ]:
rm -r mask2former

### high_quality_architectural壁＋部屋

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
import numpy as np
import cv2
import re
import subprocess
from svg.path import parse_path
import shutil

def parse_transform(transform_str):
    """transform属性を3x3行列に変換"""
    import re
    m = re.search(r"matrix\(\s*([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)\s*\)", transform_str)
    if m:
        a, b, c, d, e, f = map(float, m.groups())
        return np.array([[a, c, e],
                         [b, d, f],
                         [0, 0, 1]])
    return np.eye(3)

def transform_point(x, y, T):
    """点(x, y)に変換行列Tを適用"""
    pt = np.array([x, y, 1])
    tpt = T @ pt
    return tpt[0], tpt[1]

def extract_room_wall_masks(svg_file, output_color_file):
    """
    SVGファイルから「壁はすべて同一インスタンスID、部屋はインスタンスごとに固有のID」で
    カラーマスクを作成し、PNG形式で保存する関数
    """
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得。属性がなければスキップ
    if "width" not in root.attrib or "height" not in root.attrib:
        print(f"SVGのサイズが指定されていないため、スキップ: {svg_file}")
        return
    width = int(float(root.attrib["width"]))
    height = int(float(root.attrib["height"]))

    # インスタンスID管理用のマスク（16bitにしておくと余裕がある）
    room_instance_mask = np.zeros((height, width), dtype=np.uint16)

    # 壁 (Wall) 用の固定インスタンスID
    WALL_ID = 1
    # 部屋 (Room/Space/LivingRoom) のインスタンスID開始番号
    current_room_id = 2  # 2からスタート

    # クラス名の判定用
    wall_keywords = {"Wall", "Walls"}
    room_keywords = {"Room", "LivingRoom", "Space"}

    def fill_polygon(pts, fill_id):
        """頂点配列 pts (float座標) を fill_id で塗りつぶす"""
        pts_array = np.array(pts, dtype=np.int32)
        cv2.fillPoly(room_instance_mask, [pts_array], fill_id)

    def process_element(element, parent_class, parent_style, current_transform):
        nonlocal current_room_id
        element_class = element.attrib.get("class", "")
        classes = element_class.split() if element_class else []
        parent_tokens = parent_class.split() if parent_class else []
        all_classes = set(classes) | set(parent_tokens)

        # 壁に該当するか、部屋に該当するか判定
        is_wall = any(token in all_classes for token in wall_keywords)
        is_room = any(token in all_classes for token in room_keywords)

        # 壁でも部屋でもない場合はスキップ
        if not (is_wall or is_room):
            return

        # どのIDで塗るか
        fill_id = WALL_ID if is_wall else current_room_id

        tag = element.tag.split('}')[-1]
        d = None

        if tag == "path":
            d = element.attrib.get("d", "")
            if d:
                try:
                    path_obj = parse_path(d)
                    pts = []
                    for segment in path_obj:
                        seg_length = segment.length(error=1e-5)
                        num_samples = max(10, int(seg_length))
                        for t in np.linspace(0, 1, num_samples, endpoint=False):
                            pt = segment.point(t)
                            pts.append((float(pt.real), float(pt.imag)))

                    if pts:
                        # パスがZで閉じられている場合に最初の頂点を最後にも追加
                        if d.strip()[-1].upper() == "Z":
                            pts.append(pts[0])
                        else:
                            pt = path_obj[-1].point(1.0)
                            pts.append((float(pt.real), float(pt.imag)))

                        # 変換行列を適用
                        pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                        fill_polygon(pts_transformed, fill_id)

                except Exception as e:
                    print(f"Error processing path: {e}")

        elif tag == "polygon":
            points_str = element.attrib.get("points", "")
            try:
                point_pairs = points_str.strip().split()
                pts = []
                for pair in point_pairs:
                    coords = pair.strip().split(',')
                    if len(coords) >= 2:
                        pts.append((float(coords[0]), float(coords[1])))
                if pts:
                    pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                    fill_polygon(pts_transformed, fill_id)
            except Exception as e:
                print(f"Error processing polygon: {e}")

        elif tag == "rect":
            try:
                x = float(element.attrib.get("x", "0"))
                y = float(element.attrib.get("y", "0"))
                w = float(element.attrib.get("width", "0"))
                h = float(element.attrib.get("height", "0"))
                rect_pts = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
                pts_transformed = [transform_point(px, py, current_transform) for (px, py) in rect_pts]
                fill_polygon(pts_transformed, fill_id)
            except Exception as e:
                print(f"Error processing rect: {e}")

        # 部屋なら塗り終わった後にIDをインクリメント
        if is_room:
            current_room_id += 1

    def process_group(group, current_transform, parent_style):
        if "transform" in group.attrib:
            T = parse_transform(group.attrib["transform"])
            current_transform = current_transform @ T

        group_class = group.attrib.get("class", "")
        style = group.attrib.get("style", "") + parent_style

        for child in group:
            tag = child.tag.split('}')[-1]
            if tag == "g":
                process_group(child, current_transform, style)
            else:
                process_element(child, group_class, style, current_transform)

    # ルート要素から処理開始
    identity = np.eye(3)
    for child in root:
        tag = child.tag.split('}')[-1]
        if tag == "g":
            process_group(child, identity, "")
        else:
            process_element(child, "", "", identity)

    # --- カラーマスクを作成 ---
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)

    # すべてのユニークなIDを取り出し（0は除外）
    unique_ids = np.unique(room_instance_mask)
    unique_ids = unique_ids[unique_ids != 0]

    # 「壁」を塗る固定色 (例: 緑)
    WALL_COLOR = (0, 255, 0)  # (B, G, R)順ではなく (R, G, B) なら適宜変えてください

    # ランダム色生成時に壁色と被らないように注意
    np.random.seed(42)

    def random_color_avoid(avoid_color):
        """
        壁色と被らないランダム色を返す
        """
        while True:
            c = tuple(map(int, np.random.randint(0, 256, 3)))
            if c != avoid_color:
                return c

    for uid in unique_ids:
        mask = (room_instance_mask == uid)

        if uid == WALL_ID:
            # 壁クラス（ID=1）はすべて同じ色
            color_mask[mask] = WALL_COLOR
        else:
            # 部屋クラス（ID=2以上）はランダム色
            c = random_color_avoid(WALL_COLOR)
            color_mask[mask] = c

    # カラーマスクを保存
    cv2.imwrite(output_color_file, color_mask)
    #print(f"Saved color mask: {output_color_file}")

def svg_to_png_with_inkscape(input_svg, output_png):
    """
    Inkscape コマンドラインを使って SVG を PNG に変換するヘルパー関数。
    Inkscape 1.0 以降がインストールされている必要があります。
    """
    import subprocess
    cmd = [
        "inkscape",
        "--export-type=png",
        f"--export-filename={output_png}",
        "--export-background=#ffffff",       # 背景色を白
        "--export-background-opacity=1.0",    # 不透明度
        input_svg
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if __name__ == '__main__':
    import os
    import glob
    from tqdm import tqdm

    os.makedirs("mask2former_tmp", exist_ok=True)
    n = 0

    for base_dir in ["/Users/kohei/kkg/kkg-area-detection/sandbox/cubicasa5k/cubicasa5k/high_quality_architectural"]:
        for i in tqdm(range(1, 20230)):
            svg_file = os.path.join(base_dir, str(i), "model.svg")

            if not os.path.exists(svg_file):
                continue

            # 元画像コピー
            ori_file = os.path.join(base_dir, str(i), "F1_scaled.png")
            if not os.path.exists(ori_file):
                continue

            n += 1
            dst_img = f"mask2former_tmp/original_floorplan_{n}.png"
            shutil.copy(ori_file, dst_img)

            output_dir = "mask2former_tmp/"
            os.makedirs(output_dir, exist_ok=True)

            # カラーマスク出力先
            output_color_file = f"{output_dir}/room_wall_mask_color_{n}.png"

            # 壁＆部屋マスクを抽出して保存
            extract_room_wall_masks(svg_file, output_color_file)


  1%|          | 122/20229 [00:05<14:38, 22.88it/s]


KeyboardInterrupt: 

display: noneでもスキップしないように変更

左上基準でクロップすると解決

### high_quality_architectural部屋のみ

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
import numpy as np
import cv2
import re
import subprocess
from svg.path import parse_path
import shutil


def parse_transform(transform_str):
    """transform属性を3x3行列に変換"""
    m = re.search(r"matrix\(\s*([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)\s*\)", transform_str)
    if m:
        a, b, c, d, e, f = map(float, m.groups())
        return np.array([[a, c, e],
                         [b, d, f],
                         [0, 0, 1]])
    return np.eye(3)

def transform_point(x, y, T):
    """点(x, y)に変換行列Tを適用"""
    pt = np.array([x, y, 1])
    tpt = T @ pt
    return tpt[0], tpt[1]

def extract_room_masks(svg_file, output_file, output_color_file):
    """SVGファイルからroom_maskを抽出してPNG形式で保存"""
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得。属性がなければスルー（処理を中断）
    if "width" not in root.attrib or "height" not in root.attrib:
        print(f"SVGのサイズが指定されていないため、スキップ: {svg_file}")
        return
    width = int(float(root.attrib["width"]))
    height = int(float(root.attrib["height"]))


    # マスクの初期化
    room_mask = np.zeros((height, width), dtype=np.uint8)
    room_instance_mask = np.zeros((height, width), dtype=np.uint16)
    instance_id = 1

    # 部屋とみなすクラスのキーワード
    room_keywords = {"Room", "LivingRoom", "Space"}

    def process_element(element, parent_class, parent_style, current_transform):
        nonlocal instance_id
        element_class = element.attrib.get("class", "")
        classes = element_class.split() if element_class else []
        parent_tokens = parent_class.split() if parent_class else []
        all_classes = set(classes) | set(parent_tokens)

        # "Room", "LivingRoom", "Space" を含むかどうか判定
        if not any(token in all_classes for token in room_keywords):
            return

        # display:none の要素はスキップしない
        style = element.attrib.get("style", "") + parent_style
        #if "display:none" in style.replace(" ", ""):
        #    return

        tag = element.tag.split('}')[-1]
        #print(f"Processing room element: {tag}, classes: {all_classes}")

        if tag == "path":
            d = element.attrib.get("d", "")
            if d:
                try:
                    path_obj = parse_path(d)
                    pts = []
                    for segment in path_obj:
                        seg_length = segment.length(error=1e-5)
                        num_samples = max(10, int(seg_length))
                        for t in np.linspace(0, 1, num_samples, endpoint=False):
                            pt = segment.point(t)
                            pts.append((float(pt.real), float(pt.imag)))

                    if pts:
                        if d.strip()[-1].upper() == "Z":
                            pts.append(pts[0])
                        else:
                            pt = path_obj[-1].point(1.0)
                            pts.append((float(pt.real), float(pt.imag)))

                        pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                        pts_array = np.array(pts_transformed, dtype=np.int32)

                        fill_attr = element.attrib.get("fill", "")
                        if not (fill_attr and fill_attr.lower() == "none"):
                            cv2.fillPoly(room_mask, [pts_array], 255)
                            cv2.fillPoly(room_instance_mask, [pts_array], instance_id)
                            instance_id += 1
                except Exception as e:
                    print(f"Error processing path: {e}")

        elif tag == "polygon":
            points_str = element.attrib.get("points", "")
            try:
                point_pairs = points_str.strip().split()
                pts = []
                for pair in point_pairs:
                    coords = pair.strip().split(',')
                    if len(coords) >= 2:
                        pts.append((float(coords[0]), float(coords[1])))
                if pts:
                    pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                    pts_array = np.array(pts_transformed, dtype=np.int32)
                    cv2.fillPoly(room_mask, [pts_array], 255)
                    cv2.fillPoly(room_instance_mask, [pts_array], instance_id)
                    instance_id += 1
            except Exception as e:
                print(f"Error processing polygon: {e}")

        elif tag == "rect":
            try:
                x = float(element.attrib.get("x", "0"))
                y = float(element.attrib.get("y", "0"))
                w = float(element.attrib.get("width", "0"))
                h = float(element.attrib.get("height", "0"))
                rect_pts = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
                pts_transformed = [transform_point(px, py, current_transform) for (px, py) in rect_pts]
                pts_array = np.array(pts_transformed, dtype=np.int32)
                cv2.fillPoly(room_mask, [pts_array], 255)
                cv2.fillPoly(room_instance_mask, [pts_array], instance_id)
                instance_id += 1
            except Exception as e:
                print(f"Error processing rect: {e}")

    def process_group(group, current_transform, parent_style):
        style = group.attrib.get("style", "") + parent_style
        group_class = group.attrib.get("class", "")

        if "transform" in group.attrib:
            T = parse_transform(group.attrib["transform"])
            current_transform = current_transform @ T

        for child in group:
            tag = child.tag.split('}')[-1]
            if tag == "g":
                process_group(child, current_transform, style)
            else:
                process_element(child, group_class, style, current_transform)

    # ルート要素から処理開始
    identity = np.eye(3)
    for child in root:
        tag = child.tag.split('}')[-1]
        if tag == "g":
            process_group(child, identity, "")
        else:
            process_element(child, "", "", identity)

    # カラーマスクの生成
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
    unique_ids = np.unique(room_instance_mask)[1:]  # 0を除外

    # 各インスタンスに異なる色を割り当て
    np.random.seed(42)
    for uid in unique_ids:
        color = tuple(map(int, np.random.randint(0, 256, 3)))
        mask = room_instance_mask == uid
        color_mask[mask] = color

        # インスタンスIDを文字で重ねる例
        """
        y, x = np.where(mask)
        if len(x) > 0 and len(y) > 0:
            center_x = int(np.mean(x))
            center_y = int(np.mean(y))
            cv2.putText(color_mask, str(uid), (center_x, center_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        """

    # 結果を保存
    #cv2.imwrite(output_file, room_mask)
    cv2.imwrite(output_color_file, color_mask)
    #print(f"Saved room mask to: {output_file}")
    #print(f"Saved colored room mask to: {output_color_file}")

def svg_to_png_with_inkscape(input_svg, output_png):
    """
    Inkscape コマンドラインを使って SVG を PNG に変換するヘルパー関数。
    Inkscape 1.0 以降がインストールされている必要があります。
    """
    cmd = [
        "inkscape",
        "--export-type=png",
        f"--export-filename={output_png}",
        "--export-background=#ffffff",       # 背景色を白
        "--export-background-opacity=1.0",    # 不透明度を 1.0
        input_svg
    ]
    # subprocess.run() で外部コマンドを実行する
    subprocess.run(
        cmd,
        check=True,
        stdout=subprocess.DEVNULL,   # 必要なら標準出力も無視
        stderr=subprocess.DEVNULL    # 警告を含むエラー出力を非表示
    )

if __name__ == '__main__':
    import os
    import glob
    from tqdm import tqdm

    os.makedirs("mask2former", exist_ok=True)
    n = 0

    for base_dir in ["cubicasa5k/cubicasa5k/high_quality_architectural", "cubicasa5k/cubicasa5k/colorful", "cubicasa5k/cubicasa5k/high_quality"]:
        for i in tqdm(range(1, 20230)):
            # 該当するSVGファイルを検索
            # SVGファイルのパスを直接指定
            svg_file = os.path.join(base_dir, str(i), "model.svg")

            # ファイルが存在しない場合はスキップ
            if not os.path.exists(svg_file):
                continue
            ori_file = os.path.join(base_dir, str(i), "F1_scaled.png")

            n += 1


            shutil.copy(ori_file, f"mask2former/original_floorplan_{n}.png")

            # 出力ファイル名を設定
            output_dir = "mask2former/"
            os.makedirs(output_dir, exist_ok=True)

            output_file = f"{output_dir}/room_mask_{n}.png"
            output_color_file = f"{output_dir}/room_mask_color_{n}.png"

            # 部屋領域マスクを抽出
            extract_room_masks(svg_file, output_file, output_color_file)


In [ ]:
!cp mask2former/room_mask_color* mask2former_resize/

### high_quality_architecturalのすべてのオブジェクトをマスク化

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
import numpy as np
import cv2
import re
import subprocess
from svg.path import parse_path
import shutil
import os
import glob
from tqdm import tqdm

def parse_transform(transform_str):
    """transform属性を3x3行列に変換"""
    m = re.search(r"matrix\(\s*([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)\s*\)", transform_str)
    if m:
        a, b, c, d, e, f = map(float, m.groups())
        return np.array([[a, c, e],
                         [b, d, f],
                         [0, 0, 1]])
    return np.eye(3)

def transform_point(x, y, T):
    """点(x, y)に変換行列Tを適用"""
    pt = np.array([x, y, 1])
    tpt = T @ pt
    return tpt[0], tpt[1]

def extract_object_masks(svg_file, output_mask_file, output_color_mask_file):
    """
    SVGファイルから全ての要素（path、polygon、rectなど）を対象にマスクを抽出してPNG形式で保存。

    - output_mask_file: オブジェクト部分が255、その他は0の2値マスク
    - output_color_mask_file: オブジェクトのインスタンスごとに異なる色を付けたカラー版マスク
    """
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得。属性がなければスルー（処理を中断）
    if "width" not in root.attrib or "height" not in root.attrib:
        print(f"SVGのサイズが指定されていないため、スキップ: {svg_file}")
        return
    width = int(float(root.attrib["width"]))
    height = int(float(root.attrib["height"]))

    # マスクの初期化
    object_mask = np.zeros((height, width), dtype=np.uint8)
    object_instance_mask = np.zeros((height, width), dtype=np.uint16)
    instance_id = 1

    def process_element(element, parent_class, parent_style, current_transform):
        """
        グループ以外の要素(<path>, <polygon>, <rect>)を処理してマスクを描画する。
        """
        nonlocal instance_id

        # display:none を持つ要素を無視したい場合は下記を参照
        # style_str = (element.attrib.get("style", "") + parent_style).replace(" ", "")
        # if "display:none" in style_str:
        #    return

        tag = element.tag.split('}')[-1]

        if tag == "path":
            d = element.attrib.get("d", "")
            if d:
                try:
                    path_obj = parse_path(d)
                    pts = []
                    for segment in path_obj:
                        seg_length = segment.length(error=1e-5)
                        num_samples = max(10, int(seg_length))
                        for t in np.linspace(0, 1, num_samples, endpoint=False):
                            pt = segment.point(t)
                            pts.append((float(pt.real), float(pt.imag)))

                    if pts:
                        # パスが Z (閉じている) なら始点を終点として追加
                        if d.strip()[-1].upper() == "Z":
                            pts.append(pts[0])
                        else:
                            pt = path_obj[-1].point(1.0)
                            pts.append((float(pt.real), float(pt.imag)))

                        pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                        pts_array = np.array(pts_transformed, dtype=np.int32)

                        fill_attr = element.attrib.get("fill", "")
                        if not (fill_attr and fill_attr.lower() == "none"):
                            cv2.fillPoly(object_mask, [pts_array], 255)
                            cv2.fillPoly(object_instance_mask, [pts_array], instance_id)
                            instance_id += 1
                except Exception as e:
                    print(f"Error processing path: {e}")

        elif tag == "polygon":
            points_str = element.attrib.get("points", "")
            try:
                point_pairs = points_str.strip().split()
                pts = []
                for pair in point_pairs:
                    coords = pair.strip().split(',')
                    if len(coords) >= 2:
                        pts.append((float(coords[0]), float(coords[1])))
                if pts:
                    pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                    pts_array = np.array(pts_transformed, dtype=np.int32)
                    cv2.fillPoly(object_mask, [pts_array], 255)
                    cv2.fillPoly(object_instance_mask, [pts_array], instance_id)
                    instance_id += 1
            except Exception as e:
                print(f"Error processing polygon: {e}")

        elif tag == "rect":
            try:
                x = float(element.attrib.get("x", "0"))
                y = float(element.attrib.get("y", "0"))
                w = float(element.attrib.get("width", "0"))
                h = float(element.attrib.get("height", "0"))
                rect_pts = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
                pts_transformed = [transform_point(px, py, current_transform) for (px, py) in rect_pts]
                pts_array = np.array(pts_transformed, dtype=np.int32)
                cv2.fillPoly(object_mask, [pts_array], 255)
                cv2.fillPoly(object_instance_mask, [pts_array], instance_id)
                instance_id += 1
            except Exception as e:
                print(f"Error processing rect: {e}")

    def process_group(group, current_transform, parent_style):
        """
        <g> 要素を再帰的にめぐって、それぞれの子要素を処理する。
        """
        style = group.attrib.get("style", "") + parent_style
        group_class = group.attrib.get("class", "")

        if "transform" in group.attrib:
            T = parse_transform(group.attrib["transform"])
            current_transform = current_transform @ T

        for child in group:
            tag = child.tag.split('}')[-1]
            if tag == "g":
                # グループの場合、再帰的に処理を行う
                process_group(child, current_transform, style)
            else:
                # グループ以外の要素をマスクに反映
                process_element(child, group_class, style, current_transform)

    # ルート要素（<svg>）直下を走査
    identity = np.eye(3)
    for child in root:
        tag = child.tag.split('}')[-1]
        if tag == "g":
            process_group(child, identity, "")
        else:
            # <g> ではない要素が直下にある場合も処理
            process_element(child, "", "", identity)

    # ここから、インスタンスマスクを色付きに変換
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
    unique_ids = np.unique(object_instance_mask)
    # 0は背景なので除外
    unique_ids = unique_ids[unique_ids != 0]

    np.random.seed(42)
    for uid in unique_ids:
        color = tuple(map(int, np.random.randint(0, 256, 3)))
        mask = (object_instance_mask == uid)
        color_mask[mask] = color

    # 結果を保存
    cv2.imwrite(output_mask_file, object_mask)         # 2値マスク (グレースケール)
    cv2.imwrite(output_color_mask_file, color_mask)    # カラーインスタンスマスク

def svg_to_png_with_inkscape(input_svg, output_png):
    """
    Inkscape コマンドラインを使って SVG を PNG に変換するヘルパー関数。
    Inkscape 1.0 以降がインストールされている必要があります。
    """
    cmd = [
        "inkscape",
        "--export-type=png",
        f"--export-filename={output_png}",
        "--export-background=#ffffff",       # 背景色を白
        "--export-background-opacity=1.0",    # 不透明度を 1.0
        input_svg
    ]
    subprocess.run(
        cmd,
        check=True,
        stdout=subprocess.DEVNULL,   # 必要なら標準出力も無視
        stderr=subprocess.DEVNULL    # 警告を含むエラー出力を非表示
    )

if __name__ == '__main__':
    os.makedirs("mask2former_allobj", exist_ok=True)
    n = 0

    # 任意のディレクトリリスト
    for base_dir in ["cubicasa5k/cubicasa5k/high_quality_architectural",]:
        for i in tqdm(range(1, 20230)):
            svg_file = os.path.join(base_dir, str(i), "model.svg")
            # ファイルが存在しない場合はスキップ
            if not os.path.exists(svg_file):
                continue

            ori_file = os.path.join(base_dir, str(i), "F1_scaled.png")
            if not os.path.exists(ori_file):
                continue

            n += 1

            # オリジナル画像をコピー（バックアップ的に）
            shutil.copy(ori_file, f"mask2former_allobj/original_floorplan_{n}.png")

            # 出力ファイル名を設定
            output_dir = "mask2former_allobj/"
            os.makedirs(output_dir, exist_ok=True)

            output_mask_file = f"{output_dir}/object_mask_{n}.png"
            output_color_file = f"{output_dir}/object_mask_color_{n}.png"

            # SVG から全オブジェクトを対象にマスクを抽出
            extract_object_masks(svg_file, output_mask_file, output_color_file)


### high_quality_architectural以外の画像を前処理

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
import numpy as np
import cv2
import re
import subprocess
from svg.path import parse_path
import shutil

def parse_transform(transform_str):
    """transform属性を3x3行列に変換"""
    import re
    m = re.search(r"matrix\(\s*([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)\s*\)", transform_str)
    if m:
        a, b, c, d, e, f = map(float, m.groups())
        return np.array([[a, c, e],
                         [b, d, f],
                         [0, 0, 1]])
    return np.eye(3)

def transform_point(x, y, T):
    """点(x, y)に変換行列Tを適用"""
    pt = np.array([x, y, 1])
    tpt = T @ pt
    return tpt[0], tpt[1]

def extract_room_wall_masks(svg_file, output_color_file):
    """
    SVGファイルから「壁はすべて同一インスタンスID、部屋はインスタンスごとに固有のID」で
    カラーマスクを作成し、PNG形式で保存する関数
    """
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得。属性がなければスキップ
    if "width" not in root.attrib or "height" not in root.attrib:
        print(f"SVGのサイズが指定されていないため、スキップ: {svg_file}")
        return
    width = int(float(root.attrib["width"]))
    height = int(float(root.attrib["height"]))

    # インスタンスID管理用のマスク（16bitにしておくと余裕がある）
    room_instance_mask = np.zeros((height, width), dtype=np.uint16)

    # 壁 (Wall) 用の固定インスタンスID
    WALL_ID = 1
    # 部屋 (Room/Space/LivingRoom) のインスタンスID開始番号
    current_room_id = 2  # 2からスタート

    # クラス名の判定用
    wall_keywords = {"Wall", "Walls"}
    room_keywords = {"Room", "LivingRoom", "Space"}

    def fill_polygon(pts, fill_id):
        """頂点配列 pts (float座標) を fill_id で塗りつぶす"""
        pts_array = np.array(pts, dtype=np.int32)
        cv2.fillPoly(room_instance_mask, [pts_array], fill_id)

    def process_element(element, parent_class, parent_style, current_transform):
        nonlocal current_room_id
        element_class = element.attrib.get("class", "")
        classes = element_class.split() if element_class else []
        parent_tokens = parent_class.split() if parent_class else []
        all_classes = set(classes) | set(parent_tokens)

        # 壁に該当するか、部屋に該当するか判定
        is_wall = any(token in all_classes for token in wall_keywords)
        is_room = any(token in all_classes for token in room_keywords)

        # 壁でも部屋でもない場合はスキップ
        if not (is_wall or is_room):
            return

        # どのIDで塗るか
        fill_id = WALL_ID if is_wall else current_room_id

        tag = element.tag.split('}')[-1]
        d = None

        if tag == "path":
            d = element.attrib.get("d", "")
            if d:
                try:
                    path_obj = parse_path(d)
                    pts = []
                    for segment in path_obj:
                        seg_length = segment.length(error=1e-5)
                        num_samples = max(10, int(seg_length))
                        for t in np.linspace(0, 1, num_samples, endpoint=False):
                            pt = segment.point(t)
                            pts.append((float(pt.real), float(pt.imag)))

                    if pts:
                        # パスがZで閉じられている場合に最初の頂点を最後にも追加
                        if d.strip()[-1].upper() == "Z":
                            pts.append(pts[0])
                        else:
                            pt = path_obj[-1].point(1.0)
                            pts.append((float(pt.real), float(pt.imag)))

                        # 変換行列を適用
                        pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                        fill_polygon(pts_transformed, fill_id)

                except Exception as e:
                    print(f"Error processing path: {e}")

        elif tag == "polygon":
            points_str = element.attrib.get("points", "")
            try:
                point_pairs = points_str.strip().split()
                pts = []
                for pair in point_pairs:
                    coords = pair.strip().split(',')
                    if len(coords) >= 2:
                        pts.append((float(coords[0]), float(coords[1])))
                if pts:
                    pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                    fill_polygon(pts_transformed, fill_id)
            except Exception as e:
                print(f"Error processing polygon: {e}")

        elif tag == "rect":
            try:
                x = float(element.attrib.get("x", "0"))
                y = float(element.attrib.get("y", "0"))
                w = float(element.attrib.get("width", "0"))
                h = float(element.attrib.get("height", "0"))
                rect_pts = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
                pts_transformed = [transform_point(px, py, current_transform) for (px, py) in rect_pts]
                fill_polygon(pts_transformed, fill_id)
            except Exception as e:
                print(f"Error processing rect: {e}")

        # 部屋なら塗り終わった後にIDをインクリメント
        if is_room:
            current_room_id += 1

    def process_group(group, current_transform, parent_style):
        if "transform" in group.attrib:
            T = parse_transform(group.attrib["transform"])
            current_transform = current_transform @ T

        group_class = group.attrib.get("class", "")
        style = group.attrib.get("style", "") + parent_style

        for child in group:
            tag = child.tag.split('}')[-1]
            if tag == "g":
                process_group(child, current_transform, style)
            else:
                process_element(child, group_class, style, current_transform)

    # ルート要素から処理開始
    identity = np.eye(3)
    for child in root:
        tag = child.tag.split('}')[-1]
        if tag == "g":
            process_group(child, identity, "")
        else:
            process_element(child, "", "", identity)

    # --- カラーマスクを作成 ---
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)

    # すべてのユニークなIDを取り出し（0は除外）
    unique_ids = np.unique(room_instance_mask)
    unique_ids = unique_ids[unique_ids != 0]

    # 「壁」を塗る固定色 (例: 緑)
    WALL_COLOR = (0, 255, 0)  # (B, G, R)順ではなく (R, G, B) なら適宜変えてください

    # ランダム色生成時に壁色と被らないように注意
    np.random.seed(42)

    def random_color_avoid(avoid_color):
        """
        壁色と被らないランダム色を返す
        """
        while True:
            c = tuple(map(int, np.random.randint(0, 256, 3)))
            if c != avoid_color:
                return c

    for uid in unique_ids:
        mask = (room_instance_mask == uid)

        if uid == WALL_ID:
            # 壁クラス（ID=1）はすべて同じ色
            color_mask[mask] = WALL_COLOR
        else:
            # 部屋クラス（ID=2以上）はランダム色
            c = random_color_avoid(WALL_COLOR)
            color_mask[mask] = c

    # カラーマスクを保存
    cv2.imwrite(output_color_file, color_mask)
    #print(f"Saved color mask: {output_color_file}")

def svg_to_png_with_inkscape(input_svg, output_png):
    """
    Inkscape コマンドラインを使って SVG を PNG に変換するヘルパー関数。
    Inkscape 1.0 以降がインストールされている必要があります。
    """
    import subprocess
    cmd = [
        "inkscape",
        "--export-type=png",
        f"--export-filename={output_png}",
        "--export-background=#ffffff",       # 背景色を白
        "--export-background-opacity=1.0",    # 不透明度
        input_svg
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if __name__ == '__main__':
    import os
    import glob
    from tqdm import tqdm

    os.makedirs("mask2former", exist_ok=True)
    n = 3733

    for base_dir in ["cubicasa5k/cubicasa5k/colorful", "cubicasa5k/cubicasa5k/high_quality"]:
        for i in tqdm(range(1, 20230)):
            svg_file = os.path.join(base_dir, str(i), "model.svg")

            if not os.path.exists(svg_file):
                continue

            # 元画像コピー
            ori_file = os.path.join(base_dir, str(i), "F1_scaled.png")
            if not os.path.exists(ori_file):
                continue

            n += 1
            dst_img = f"mask2former/original_floorplan_{n}.png"
            shutil.copy(ori_file, dst_img)

            output_dir = "mask2former/"
            os.makedirs(output_dir, exist_ok=True)

            # カラーマスク出力先
            output_color_file = f"{output_dir}/room_wall_mask_color_{n}.png"

            # 壁＆部屋マスクを抽出して保存
            extract_room_wall_masks(svg_file, output_color_file)


In [ ]:
import cv2
import os
from tqdm import tqdm

def align_resize_and_crop(input_dir, output_dir):
    """
    フロアプラン画像 (original_floorplan_{i}.png) と
    マスク画像 (room_mask_color_{i}.png) を以下の手順で処理する:
      1. 2つの画像の幅のうち、小さい方の幅に合わせて大きい方を縮小する。
         - 幅の比率を基準に各画像を縮小（小さい画像はそのまま）。
      2. 縮小後、両画像の高さが異なる場合は、左上を基準に
         高さが小さい方のサイズにクロップする。
    最終的に、両画像は同じサイズとなる。
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in tqdm(range(1, 6001)):
        floorplan_path = os.path.join(input_dir, f"original_floorplan_{i}.png")
        mask_path      = os.path.join(input_dir, f"room_wall_mask_color_{i}.png")

        if not (os.path.exists(floorplan_path) and os.path.exists(mask_path)):
            continue

        floorplan = cv2.imread(floorplan_path)
        mask_img  = cv2.imread(mask_path)

        if floorplan is None or mask_img is None:
            continue

        # 各画像の幅・高さを取得
        h_f, w_f = floorplan.shape[:2]
        h_m, w_m = mask_img.shape[:2]

        # 2つの画像のうち、小さい幅を基準にする
        target_width = min(w_f, w_m)

        # 小さい方を採用して左上基準でクロップ
        target_height = min(h_f, h_m)
        floorplan_cropped = floorplan[0:target_height, 0:target_width]
        mask_cropped      = mask_img[0:target_height, 0:target_width]

        # 出力ファイルパス
        out_floorplan = os.path.join(output_dir, f"original_floorplan_{i}.png")
        out_mask      = os.path.join(output_dir, f"room_mask_color_{i}.png")

        cv2.imwrite(out_floorplan, floorplan_cropped)
        cv2.imwrite(out_mask, mask_cropped)

if __name__ == "__main__":
    input_folder = "mask2former"        # 元画像が置いてあるフォルダ
    output_folder = "mask2former_resize" # 出力先フォルダ

    align_resize_and_crop(input_folder, output_folder)

### high_quality_architecturalのオリジナル画像の背景を白に

In [ ]:
import cv2
import os
from tqdm import tqdm

def align_resize_and_crop(input_dir, output_dir):
    """
    フロアプラン画像 (original_floorplan_{i}.png) と
    マスク画像 (room_mask_color_{i}.png) を以下の手順で処理する:
      1. 2つの画像の幅のうち、小さい方の幅に合わせて大きい方を縮小する。
         - 幅の比率を基準に各画像を縮小（小さい画像はそのまま）。
      2. 縮小後、両画像の高さが異なる場合は、左上を基準に
         高さが小さい方のサイズにクロップする。
    最終的に、両画像は同じサイズとなる。
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in tqdm(range(1, 6001)):
        #room_mask_color
        #room_wall_mask_color
        floorplan_path = os.path.join(input_dir, f"original_floorplan_{i}.png")
        mask_path      = os.path.join(input_dir, f"room_wall_mask_color_{i}.png")
        mask_allobj      = os.path.join(f"mask2former_allobj/object_mask_{i}.png")

        if not (os.path.exists(floorplan_path) and os.path.exists(mask_path)):
            print(f"Skipping {floorplan_path} and {mask_path}")
            continue

        floorplan = cv2.imread(floorplan_path)
        mask_img  = cv2.imread(mask_path)
        mask_allobj  = cv2.imread(mask_allobj)

        if floorplan is None or mask_img is None:
            print(f"Failed to load image: {floorplan_path}")
            continue

        # 各画像の幅・高さを取得
        h_f, w_f = floorplan.shape[:2]
        h_m, w_m = mask_img.shape[:2]

        # 2つの画像のうち、小さい幅を基準にする
        target_width = min(w_f, w_m)

        # 小さい方を採用して左上基準でクロップ
        target_height = min(h_f, h_m)
        floorplan_cropped = floorplan[0:target_height, 0:target_width]
        mask_cropped      = mask_img[0:target_height, 0:target_width]
        mask_allobj_cropped      = mask_allobj[0:target_height, 0:target_width]

        # 出力ファイルパス
        out_floorplan = os.path.join(output_dir, f"original_floorplan_{i}.png")
        out_mask      = os.path.join(output_dir, f"room_mask_color_{i}.png")
        out_mask_allobj      = os.path.join(output_dir, f"room_mask_color_allobj_{i}.png")

        cv2.imwrite(out_floorplan, floorplan_cropped)
        cv2.imwrite(out_mask, mask_cropped)
        cv2.imwrite(out_mask_allobj, mask_allobj_cropped)

        # === ここから追加: マスクを使ってオブジェクト外を白塗り ===
        # room_mask を読み込み
        mask = cv2.imread(out_mask_allobj, cv2.IMREAD_GRAYSCALE)
        ori_img = cv2.imread(out_floorplan, cv2.IMREAD_COLOR)

        # マスクが0の部分(=部屋でない領域)を白で塗りつぶす
        # mask が 255 の領域 → 部屋, 0 の領域 → 部屋外
        white_area = (mask == 0)
        ori_img[white_area] = [255, 255, 255]  # BGR 全て255で白

        # 新たなファイル名で保存
        output_white_file = f"{output_dir}/white_original_floorplan_{i}.png"
        cv2.imwrite(output_white_file, ori_img)
        # === 追加ここまで ===


if __name__ == "__main__":
    input_folder = "mask2former"        # 元画像が置いてあるフォルダ
    output_folder = "mask2former_resize" # 出力先フォルダ

    align_resize_and_crop(input_folder, output_folder)

In [ ]:
!rm -r mask2former_resize

In [ ]:
!zip -rq resize_architectural_sunpou.zip mask2former_resize2

In [ ]:
!cp resize_architectural_sunpou.zip /content/drive/MyDrive/models/resize_architecturalANDnormal_sunpou.zip

In [ ]:
!unzip -q /content/drive/MyDrive/models/resize_architectural_wall.zip

In [ ]:
! ls mask2former_resize/original_floorplan* | wc -l

In [ ]:
!zip -rq mask2former.zip mask2former

### 寸法線を入れる

In [ ]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import re
import sys
import argparse
from pathlib import Path
from svg.path import parse_path
from collections import defaultdict

# SVG名前空間
SVG_NS = {'svg': 'http://www.w3.org/2000/svg'}
ET.register_namespace('', 'http://www.w3.org/2000/svg')
ET.register_namespace('xlink', 'http://www.w3.org/1999/xlink')

def parse_transform(transform_str):
    """transform属性を3x3行列に変換"""
    m = re.search(r"matrix\(\s*([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)\s*\)", transform_str)
    if m:
        a, b, c, d, e, f = map(float, m.groups())
        return np.array([[a, c, e],
                         [b, d, f],
                         [0, 0, 1]])
    return np.eye(3)

def transform_point(x, y, T):
    """点(x, y)に変換行列Tを適用"""
    pt = np.array([x, y, 1])
    tpt = T @ pt
    return tpt[0], tpt[1]

def extract_walls_and_rooms(svg_file):
    """
    SVGファイルから壁と部屋の情報を抽出する関数

    Returns:
        tuple: (外部壁ポリゴン, 内部壁ポリゴン, 部屋ポリゴン)
    """
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得
    if "width" not in root.attrib or "height" not in root.attrib:
        print(f"SVGのサイズが指定されていないため、スキップ: {svg_file}")
        return [], [], []

    width = int(float(root.attrib["width"]))
    height = int(float(root.attrib["height"]))


    # 結果を格納するリスト
    external_walls = []  # 外部壁
    internal_walls = []  # 内部壁
    rooms = []           # 部屋

    # 部屋と壁を識別するキーワード
    room_keywords = {"Room", "LivingRoom", "Space"}
    wall_keywords = {"Wall"}

    def extract_polygon_points(element, current_transform):
        """要素からポリゴンの頂点を抽出"""
        tag = element.tag.split('}')[-1]

        if tag == "path":
            d = element.attrib.get("d", "")
            if d:
                try:
                    path_obj = parse_path(d)
                    pts = []
                    for segment in path_obj:
                        seg_length = segment.length(error=1e-5)
                        num_samples = max(10, int(seg_length))
                        for t in np.linspace(0, 1, num_samples, endpoint=False):
                            pt = segment.point(t)
                            pts.append((float(pt.real), float(pt.imag)))
                    if pts:
                        if d.strip()[-1].upper() == "Z":
                            pts.append(pts[0])
                        else:
                            pt = path_obj[-1].point(1.0)
                            pts.append((float(pt.real), float(pt.imag)))
                        pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                        return pts_transformed
                except Exception as e:
                    print(f"Error processing path: {e}")

        elif tag == "polygon":
            points_str = element.attrib.get("points", "")
            try:
                point_pairs = points_str.strip().split()
                pts = []
                for pair in point_pairs:
                    coords = pair.strip().split(',')
                    if len(coords) >= 2:
                        pts.append((float(coords[0]), float(coords[1])))
                if pts:
                    pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                    return pts_transformed
            except Exception as e:
                print(f"Error processing polygon: {e}")

        elif tag == "rect":
            try:
                x = float(element.attrib.get("x", "0"))
                y = float(element.attrib.get("y", "0"))
                w = float(element.attrib.get("width", "0"))
                h = float(element.attrib.get("height", "0"))
                rect_pts = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
                pts_transformed = [transform_point(px, py, current_transform) for (px, py) in rect_pts]
                return pts_transformed
            except Exception as e:
                print(f"Error processing rect: {e}")

        return None

    def process_element(element, parent_class, parent_style, current_transform):
        element_class = element.attrib.get("class", "")
        classes = element_class.split() if element_class else []
        parent_tokens = parent_class.split() if parent_class else []
        all_classes = set(classes) | set(parent_tokens)

        # 部屋と壁を識別
        is_room = any(token in all_classes for token in room_keywords)
        is_external_wall = any(token in all_classes for token in wall_keywords) and "External" in element_class
        is_internal_wall = any(token in all_classes for token in wall_keywords) and "External" not in element_class

        if not (is_room or is_external_wall or is_internal_wall):
            return

        # ポリゴンを抽出
        points = extract_polygon_points(element, current_transform)
        if points:
            if is_external_wall:
                external_walls.append(points)
            elif is_internal_wall:
                internal_walls.append(points)
            elif is_room:
                room_info = {
                    'points': points,
                    'id': element.attrib.get('id', ''),
                    'class': element_class
                }
                rooms.append(room_info)

    def process_group(group, current_transform, parent_style):
        style = group.attrib.get("style", "") + parent_style
        group_class = group.attrib.get("class", "")

        if "transform" in group.attrib:
            T = parse_transform(group.attrib["transform"])
            current_transform = current_transform @ T

        for child in group:
            tag = child.tag.split('}')[-1]
            if tag == "g":
                process_group(child, current_transform, style)
            else:
                process_element(child, group_class, style, current_transform)

    # ルート要素から処理開始
    identity = np.eye(3)
    for child in root:
        tag = child.tag.split('}')[-1]
        if tag == "g":
            process_group(child, identity, "")
        else:
            process_element(child, "", "", identity)

    return external_walls, internal_walls, rooms

def extract_wall_segments(walls, tolerance=1.0):
    """壁のポリゴンから水平/垂直のセグメントを抽出"""
    horizontal_segments = []
    vertical_segments = []

    for wall in walls:
        # 少なくとも2つの点が必要
        if len(wall) < 2:
            continue

        # 隣接する点のペアをチェック
        for i in range(len(wall) - 1):
            x1, y1 = wall[i]
            x2, y2 = wall[i + 1]

            # 短すぎるセグメントは無視
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            if length < 5:  # 5ピクセル未満は無視
                continue

            # 水平のセグメント（y座標がほぼ同じ）
            if abs(y1 - y2) < tolerance and abs(x1 - x2) > tolerance:
                # 左から右への順序にする
                x_left = min(x1, x2)
                x_right = max(x1, x2)
                y_avg = (y1 + y2) / 2  # y座標の平均値
                horizontal_segments.append((x_left, y_avg, x_right, y_avg))

            # 垂直のセグメント（x座標がほぼ同じ）
            if abs(x1 - x2) < tolerance and abs(y1 - y2) > tolerance:
                # 上から下への順序にする
                y_top = min(y1, y2)
                y_bottom = max(y1, y2)
                x_avg = (x1 + x2) / 2  # x座標の平均値
                vertical_segments.append((x_avg, y_top, x_avg, y_bottom))

    return horizontal_segments, vertical_segments

def merge_close_points(points, threshold):
    """近接した点をマージ"""
    if not points:
        return []

    # ソート済みの点リスト
    sorted_points = sorted(points)
    merged = [sorted_points[0]]

    for point in sorted_points[1:]:
        if point - merged[-1] > threshold:
            merged.append(point)

    return merged

def add_external_dimensions(svg_file, output_file, margin=50, min_length=20):
    """
    SVGフロアプランに外部寸法線を追加する

    Args:
        svg_file: 入力SVGファイルパス
        output_file: 出力SVGファイルパス
        margin: フロアプラン外枠と寸法線の間の余白（ピクセル）
        min_length: セグメントの最小長さ（これより短いセグメントは無視）
    """
    # SVGファイルから壁と部屋の情報を抽出
    external_walls, internal_walls, rooms = extract_walls_and_rooms(svg_file)

    # SVGファイルを解析
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得
    width = float(root.attrib['width'])
    height = float(root.attrib['height'])
    viewBox = root.attrib.get('viewBox', f"0 0 {width} {height}")
    viewBox_values = [float(x) for x in viewBox.split()]

    # 全ての壁を結合
    all_walls = external_walls + internal_walls

    # 水平・垂直の壁セグメントを抽出
    horizontal_segments, vertical_segments = extract_wall_segments(all_walls)

    # セグメントを長さでフィルタリング
    horizontal_segments = [s for s in horizontal_segments if abs(s[2] - s[0]) >= min_length]
    vertical_segments = [s for s in vertical_segments if abs(s[3] - s[1]) >= min_length]

    # 境界ボックスを計算
    all_points = []
    for wall in all_walls:
        all_points.extend(wall)

    if not all_points:
        print("壁や部屋の要素が見つかりません。")
        return

    min_x = min(p[0] for p in all_points)
    max_x = max(p[0] for p in all_points)
    min_y = min(p[1] for p in all_points)
    max_y = max(p[1] for p in all_points)

    """
    # SVGのサイズを拡張
    extended_margin = 40*2
    new_width = width + extended_margin * 2
    new_height = height + extended_margin * 2

    # SVGのビューボックスを更新
    root.attrib['width'] = str(new_width)
    root.attrib['height'] = str(new_height)

    # ビューボックスが指定されている場合は更新
    if 'viewBox' in root.attrib:
        view_min_x = viewBox_values[0] - extended_margin
        view_min_y = viewBox_values[1] - extended_margin
        view_width = viewBox_values[2] + extended_margin * 2
        view_height = viewBox_values[3] + extended_margin * 2
        root.attrib['viewBox'] = f"{view_min_x} {view_min_y} {view_width} {view_height}"
    else:
        # ビューボックスが指定されていない場合は追加
        root.attrib['viewBox'] = f"-{extended_margin} -{extended_margin} {new_width} {new_height}"
    """

    # 壁セグメントのX座標を抽出
    x_coords = []
    for segment in vertical_segments:
        midpoint_x = segment[0]  # 垂直壁のx座標
        x_coords.append(midpoint_x)

    # 壁セグメントのY座標を抽出
    y_coords = []
    for segment in horizontal_segments:
        midpoint_y = segment[1]  # 水平壁のy座標
        y_coords.append(midpoint_y)

    # 近接した座標をマージして境界を追加
    unique_x_coords = merge_close_points(x_coords, width//30)
    unique_y_coords = merge_close_points(y_coords, height//30)

    # 最終的な区切り位置を計算
    all_x_coords = [min_x] + unique_x_coords + [max_x]
    all_y_coords = [min_y] + unique_y_coords + [max_y]

    # X方向のセグメント長さを計算
    x_segments = []
    for i in range(1, len(all_x_coords)):
        length = all_x_coords[i] - all_x_coords[i-1]
        x_segments.append(round(length))

    # Y方向のセグメント長さを計算
    y_segments = []
    for i in range(1, len(all_y_coords)):
        length = all_y_coords[i] - all_y_coords[i-1]
        y_segments.append(round(length))

    # 相対座標に変換
    x_divisions = [x - min_x for x in all_x_coords]
    y_divisions = [y - min_y for y in all_y_coords]

    # 寸法線用のグループを作成
    dimension_group = ET.SubElement(root, 'g', {
        'id': 'ExternalDimensions',
        'class': 'ExternalDimensions',
        'fill': 'none',
        'stroke': '#000000',
        'stroke-width': '1'
    })

    # 寸法線を描画するヘルパー関数
    def draw_dimension_line(start_point, end_point, divisions, measurements, is_horizontal=True, text_offset=5, add_total=True, total_text=None):
        """
        寸法線を描画する

        Args:
            start_point: 開始点 (x, y)
            end_point: 終了点 (x, y)
            divisions: 区切り位置のリスト（0からの相対位置）
            measurements: 各セグメントの寸法値
            is_horizontal: 水平または垂直の寸法線
            text_offset: テキストのオフセット
            add_total: 合計寸法を表示するかどうか
            total_text: 合計寸法のカスタムテキスト
        """
        line_group = ET.SubElement(dimension_group, 'g', {'class': 'DimensionLine'})

        # メインライン
        ET.SubElement(line_group, 'line', {
            'x1': str(start_point[0]),
            'y1': str(start_point[1]),
            'x2': str(end_point[0]),
            'y2': str(end_point[1]),
            'stroke': '#000000',
            'stroke-width': '1'
        })

        # 区切り線とテキスト
        tick_length = 10

        # 全ての区切り位置にティックを描画
        for i, div_pos in enumerate(divisions):
            # 現在の位置
            if is_horizontal:
                curr_pos = (start_point[0] + div_pos, start_point[1])
            else:
                curr_pos = (start_point[0], start_point[1] + div_pos)

            # ティックマーク
            if is_horizontal:
                ET.SubElement(line_group, 'line', {
                    'x1': str(curr_pos[0]),
                    'y1': str(curr_pos[1] - tick_length),
                    'x2': str(curr_pos[0]),
                    'y2': str(curr_pos[1] + tick_length),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 最初と最後以外の区切り位置に区切り線を内部に延長
                if 0 < i < len(divisions) - 1:
                    extension_length = 15
                    if start_point[1] < min_y:  # 上部の寸法線
                        ET.SubElement(line_group, 'line', {
                            'x1': str(curr_pos[0]),
                            'y1': str(curr_pos[1]),
                            'x2': str(curr_pos[0]),
                            'y2': str(curr_pos[1] + extension_length),
                            'stroke': '#000000',
                            'stroke-width': '1',
                            'stroke-dasharray': '4,2'
                        })
                    else:  # 下部の寸法線
                        ET.SubElement(line_group, 'line', {
                            'x1': str(curr_pos[0]),
                            'y1': str(curr_pos[1]),
                            'x2': str(curr_pos[0]),
                            'y2': str(curr_pos[1] - extension_length),
                            'stroke': '#000000',
                            'stroke-width': '1',
                            'stroke-dasharray': '4,2'
                        })
            else:
                ET.SubElement(line_group, 'line', {
                    'x1': str(curr_pos[0] - tick_length),
                    'y1': str(curr_pos[1]),
                    'x2': str(curr_pos[0] + tick_length),
                    'y2': str(curr_pos[1]),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 最初と最後以外の区切り位置に区切り線を内部に延長
                if 0 < i < len(divisions) - 1:
                    extension_length = 15
                    if start_point[0] < min_x:  # 左側の寸法線
                        ET.SubElement(line_group, 'line', {
                            'x1': str(curr_pos[0]),
                            'y1': str(curr_pos[1]),
                            'x2': str(curr_pos[0] + extension_length),
                            'y2': str(curr_pos[1]),
                            'stroke': '#000000',
                            'stroke-width': '1',
                            'stroke-dasharray': '4,2'
                        })
                    else:  # 右側の寸法線
                        ET.SubElement(line_group, 'line', {
                            'x1': str(curr_pos[0]),
                            'y1': str(curr_pos[1]),
                            'x2': str(curr_pos[0] - extension_length),
                            'y2': str(curr_pos[1]),
                            'stroke': '#000000',
                            'stroke-width': '1',
                            'stroke-dasharray': '4,2'
                        })

        # 各セグメントのラベルを追加
        for i, measurement in enumerate(measurements):
            if measurement == 0:
                continue
            # 現在のセグメントの開始位置と終了位置
            start_pos = divisions[i]
            end_pos = divisions[i + 1]
            mid_pos = (start_pos + end_pos) / 2

            # テキスト位置
            if is_horizontal:
                text_x = start_point[0] + mid_pos
                text_y = start_point[1] + text_offset * (-1 if start_point[1] < min_y else 1)
                text_anchor = 'middle'

                # 水平の場合は通常のテキスト
                text_element = ET.SubElement(line_group, 'text', {
                    'x': str(text_x),
                    'y': str(text_y),
                    'font-family': 'Verdana',
                    'font-size': '18',
                    'text-anchor': text_anchor,
                    'dominant-baseline': 'middle',
                    'fill': '#000000'
                })
                text_element.text = str(measurement)
            else:
                # 垂直の場合はテキストを90度回転
                text_x = start_point[0] + text_offset * (-1 if start_point[0] < min_x else 1)
                text_y = start_point[1] + mid_pos

                # テキスト要素のグループを作成して回転させる
                text_group = ET.SubElement(line_group, 'g', {
                    'transform': f'translate({text_x},{text_y}) rotate({-90 if start_point[0] < min_x else 90})'
                })

                # 回転したテキスト要素
                text_element = ET.SubElement(text_group, 'text', {
                    'x': '0',
                    'y': '0',
                    'font-family': 'Verdana',
                    'font-size': '18',
                    'text-anchor': 'middle',
                    'dominant-baseline': 'middle',
                    'fill': '#000000'
                })
                text_element.text = str(measurement)

        # 合計寸法を表示（外側に囲むように線を引く）
        if add_total:
            # 合計値がない場合は計算
            if not total_text:
                total_value = sum(measurements)
                total_text = str(total_value)

            # 合計寸法用のグループ
            total_group = ET.SubElement(line_group, 'g', {'class': 'TotalDimension'})

            # 位置とオフセットを計算
            total_offset = 40  # 合計寸法線の外側へのオフセット

            if is_horizontal:
                # 水平方向の合計寸法線
                # 左側のティック
                ET.SubElement(total_group, 'line', {
                    'x1': str(start_point[0]),
                    'y1': str(start_point[1] - total_offset if start_point[1] < min_y else start_point[1] + total_offset),
                    'x2': str(start_point[0]),
                    'y2': str(start_point[1]),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 右側のティック
                ET.SubElement(total_group, 'line', {
                    'x1': str(end_point[0]),
                    'y1': str(end_point[1] - total_offset if end_point[1] < min_y else end_point[1] + total_offset),
                    'x2': str(end_point[0]),
                    'y2': str(end_point[1]),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 合計寸法線
                ET.SubElement(total_group, 'line', {
                    'x1': str(start_point[0]),
                    'y1': str(start_point[1] - total_offset if start_point[1] < min_y else start_point[1] + total_offset),
                    'x2': str(end_point[0]),
                    'y2': str(end_point[1] - total_offset if end_point[1] < min_y else end_point[1] + total_offset),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 合計寸法テキスト
                text_x = (start_point[0] + end_point[0]) / 2
                text_y = start_point[1] - total_offset * 1.3 if start_point[1] < min_y else start_point[1] + total_offset * 1.3

                # 水平の合計寸法テキスト
                total_element = ET.SubElement(total_group, 'text', {
                    'x': str(text_x),
                    'y': str(text_y),
                    'font-family': 'Verdana',
                    'font-size': '24',
                    'font-weight': 'bold',
                    'text-anchor': 'middle',
                    'dominant-baseline': 'middle',
                    'fill': '#000000'
                })
                total_element.text = total_text


            else:
                # 垂直方向の合計寸法線
                # 上側のティック
                ET.SubElement(total_group, 'line', {
                    'x1': str(start_point[0] - total_offset if start_point[0] < min_x else start_point[0] + total_offset),
                    'y1': str(start_point[1]),
                    'x2': str(start_point[0]),
                    'y2': str(start_point[1]),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 下側のティック
                ET.SubElement(total_group, 'line', {
                    'x1': str(end_point[0] - total_offset if end_point[0] < min_x else end_point[0] + total_offset),
                    'y1': str(end_point[1]),
                    'x2': str(end_point[0]),
                    'y2': str(end_point[1]),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 合計寸法線
                ET.SubElement(total_group, 'line', {
                    'x1': str(start_point[0] - total_offset if start_point[0] < min_x else start_point[0] + total_offset),
                    'y1': str(start_point[1]),
                    'x2': str(end_point[0] - total_offset if end_point[0] < min_x else end_point[0] + total_offset),
                    'y2': str(end_point[1]),
                    'stroke': '#000000',
                    'stroke-width': '1'
                })

                # 合計寸法テキスト
                text_x = start_point[0] - total_offset * 1.3 if start_point[0] < min_x else start_point[0] + total_offset * 1.3
                text_y = (start_point[1] + end_point[1]) / 2
                text_anchor = 'end' if start_point[0] < min_x else 'start'

                # テキストグループを作成（回転用）
                text_group = ET.SubElement(total_group, 'g', {
                    'transform': f'translate({text_x},{text_y}) rotate({-90 if start_point[0] < min_x else 90})'
                })

                # 垂直の合計寸法テキスト（横倒し）
                total_element = ET.SubElement(text_group, 'text', {
                    'x': '0',
                    'y': '0',
                    'font-family': 'Verdana',
                    'font-size': '24',
                    'font-weight': 'bold',
                    'text-anchor': 'middle',
                    'dominant-baseline': 'middle',
                    'fill': '#000000'
                })
                total_element.text = total_text

    # --- 寸法線の描画 ---

    # 上部の寸法線（水平）
    top_y = min_y - margin
    top_total = sum(x_segments)
    draw_dimension_line(
        (min_x, top_y), (max_x, top_y),
        x_divisions, x_segments,
        is_horizontal=True, text_offset=10, total_text=str(top_total)
    )

    # 下部の寸法線（水平）
    bottom_y = max_y + margin
    draw_dimension_line(
        (min_x, bottom_y), (max_x, bottom_y),
        x_divisions, x_segments,
        is_horizontal=True, text_offset=10, total_text=str(top_total)
    )

    # 左側の寸法線（垂直）
    left_x = min_x - margin
    left_total = sum(y_segments)
    draw_dimension_line(
        (left_x, min_y), (left_x, max_y),
        y_divisions, y_segments,
        is_horizontal=False, text_offset=10, total_text=str(left_total)
    )

    # 右側の寸法線（垂直）
    right_x = max_x + margin
    draw_dimension_line(
        (right_x, min_y), (right_x, max_y),
        y_divisions, y_segments,
        is_horizontal=False, text_offset=10, total_text=str(left_total)
    )

    # SVGを保存
    tree.write(output_file, encoding='utf-8', xml_declaration=True)
    print(f"外部寸法線を追加したSVGを保存しました: {output_file}")


def main():
    n = 0
    for base_dir in ["cubicasa5k/cubicasa5k/high_quality_architectural", "cubicasa5k/cubicasa5k/colorful", "cubicasa5k/cubicasa5k/high_quality"]:
        for i in tqdm(range(1, 20230)):
            svg_file = os.path.join(base_dir, str(i), "model.svg")

            if not os.path.exists(svg_file):
                continue

            # 元画像コピー

            n += 1
            output_path = f"svg/{n}.svg"

            add_external_dimensions(svg_file, output_path, 50)

if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import os
import argparse
from pathlib import Path
import subprocess
import tempfile

# SVG名前空間
SVG_NS = {'svg': 'http://www.w3.org/2000/svg'}
ET.register_namespace('', 'http://www.w3.org/2000/svg')
ET.register_namespace('xlink', 'http://www.w3.org/1999/xlink')

def create_dimension_only_svg(svg_file, temp_svg_path):
    """
    寸法線のみを含む新しいSVGファイルを作成する

    Args:
        svg_file: 元のSVGファイルパス
        temp_svg_path: 出力する一時SVGファイルパス
    """
    # 元のSVGを解析
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズと属性を取得
    width = root.attrib.get('width', '1000')
    height = root.attrib.get('height', '1000')
    viewBox = root.attrib.get('viewBox', f"0 0 {width} {height}")

    # 新しいSVG要素を作成
    new_root = ET.Element('svg', {
        'xmlns': 'http://www.w3.org/2000/svg',
        'width': width,
        'height': height,
        'viewBox': viewBox
    })

    # 背景用の白い矩形を追加
    ET.SubElement(new_root, 'rect', {
        'width': '100%',
        'height': '100%',
        'fill': 'white'
    })

    # 寸法線グループを探す
    dimension_group = None
    for element in root.findall(".//svg:g[@id='ExternalDimensions']", SVG_NS):
        dimension_group = element
        break

    if dimension_group is None:
        for element in root.findall(".//svg:g[@class='ExternalDimensions']", SVG_NS):
            dimension_group = element
            break

    # 寸法線グループが見つかった場合、新しいSVGに追加
    if dimension_group is not None:
        new_root.append(dimension_group)
    else:
        print("警告: 寸法線グループが見つかりませんでした。")

    # 新しいSVGツリーを作成
    new_tree = ET.ElementTree(new_root)

    # 一時ファイルに保存
    new_tree.write(temp_svg_path, encoding='utf-8', xml_declaration=True)

def extract_dimension_lines_to_png(svg_file, output_png, background_color="white"):
    """
    SVGから寸法線のみを抽出してPNG形式で出力する

    Args:
        svg_file: 入力SVGファイルパス
        output_png: 出力PNGファイルパス
        background_color: 背景色
    """
    # 一時SVGファイルを作成
    temp_svg = tempfile.NamedTemporaryFile(suffix='.svg', delete=False)
    temp_svg_path = temp_svg.name
    temp_svg.close()

    try:
        # 寸法線のみを含むSVGを作成
        create_dimension_only_svg(svg_file, temp_svg_path)

        # SVGをPNGに変換
        try:
            # 方法1: Inkscapeを使用（インストールされている場合）
            try:
                cmd = [
                    "inkscape",
                    "--export-type=png",
                    f"--export-filename={output_png}",
                    f"--export-background={background_color}",
                    "--export-background-opacity=1.0",
                    temp_svg_path
                ]
                subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                print(f"Inkscapeを使用して寸法線のみのPNGを出力しました: {output_png}")
                return
            except (subprocess.SubprocessError, FileNotFoundError):
                print("Inkscapeが見つからないか、エラーが発生しました。別の方法を試します。")

            # 方法2: cairosvgを使用（インストールされている場合）
            try:
                import cairosvg
                cairosvg.svg2png(url=temp_svg_path, write_to=output_png, background_color=background_color)
                print(f"cairosvgを使用して寸法線のみのPNGを出力しました: {output_png}")
                return
            except ImportError:
                print("cairosvgがインストールされていません。別の方法を試します。")

            # 方法3: rsvgを使用（インストールされている場合）
            try:
                cmd = [
                    "rsvg-convert",
                    "-f", "png",
                    "-o", output_png,
                    temp_svg_path
                ]
                subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                print(f"rsvg-convertを使用して寸法線のみのPNGを出力しました: {output_png}")
                return
            except (subprocess.SubprocessError, FileNotFoundError):
                print("rsvg-convertが見つからないか、エラーが発生しました。")

            # 方法4: svglib+reportlabを使用（インストールされている場合）
            try:
                from svglib.svglib import svg2rlg
                from reportlab.graphics import renderPM

                drawing = svg2rlg(temp_svg_path)
                renderPM.drawToFile(drawing, output_png, fmt="PNG", bg=background_color)
                print(f"svglib+reportlabを使用して寸法線のみのPNGを出力しました: {output_png}")
                return
            except ImportError:
                print("svglib+reportlabがインストールされていません。")

            # すべての方法が失敗した場合
            print("警告: SVGをPNGに変換するためのライブラリがインストールされていません。")
            print("以下のいずれかをインストールしてください:")
            print("- Inkscape (https://inkscape.org/)")
            print("- cairosvg (pip install cairosvg)")
            print("- rsvg-convert (librsvg2-bin パッケージ)")
            print("- svglib & reportlab (pip install svglib reportlab)")

            # 代替として空のPNGを作成
            blank_image = np.ones((int(float(root.attrib.get('height', '1000'))),
                                 int(float(root.attrib.get('width', '1000'))), 3),
                                np.uint8) * 255
            cv2.imwrite(output_png, blank_image)
            print(f"代替として白いPNGを出力しました: {output_png}")

        except Exception as e:
            print(f"エラー: PNG変換中に例外が発生しました: {e}")
            # 代替として空のPNGを作成
            blank_image = np.ones((1000, 1000, 3), np.uint8) * 255
            cv2.imwrite(output_png, blank_image)
    finally:
        # 一時ファイルを削除
        if os.path.exists(temp_svg_path):
            os.unlink(temp_svg_path)

def save_svg_as_png_with_dimensions_only(input_svg, output_png):
    """
    SVGを元に寸法線のみのPNGを作成する代替方法

    Args:
        input_svg: 入力SVGファイルパス
        output_png: 出力PNGファイルパス
    """
    # 寸法線要素のみを含む新しいSVGを作成
    tree = ET.parse(input_svg)
    root = tree.getroot()

    # SVGのサイズを取得
    width = int(float(root.attrib.get('width')))
    height = int(float(root.attrib.get('height')))

    # 空白画像を作成
    image = np.ones((height, width, 3), dtype=np.uint8) * 255

    # 寸法線を描画
    # 寸法線グループを探す
    dimension_elements = []

    # ExternalDimensions IDを持つ要素を検索
    for element in root.findall(".//svg:g[@id='ExternalDimensions']", SVG_NS):
        # 線と文字を探す
        for line in element.findall(".//svg:line", SVG_NS):
            dimension_elements.append(('line', line))
        for text in element.findall(".//svg:text", SVG_NS):
            dimension_elements.append(('text', text))

    # 寸法線要素が見つかった場合、描画
    for elem_type, elem in dimension_elements:
        if elem_type == 'line':
            try:
                x1 = int(float(elem.attrib.get('x1', '0')))
                y1 = int(float(elem.attrib.get('y1', '0')))
                x2 = int(float(elem.attrib.get('x2', '0')))
                y2 = int(float(elem.attrib.get('y2', '0')))

                cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 1)
            except Exception as e:
                print(f"線の描画中にエラー: {e}")

        elif elem_type == 'text':
            try:
                x = int(float(elem.attrib.get('x', '0')))
                y = int(float(elem.attrib.get('y', '0')))
                text = elem.text or ""
                font = cv2.FONT_HERSHEY_SIMPLEX
                scale = 0.5

                cv2.putText(image, text, (x, y), font, scale, (0, 0, 0), 1, cv2.LINE_AA)
            except Exception as e:
                print(f"テキストの描画中にエラー: {e}")

    # 画像を保存
    cv2.imwrite(output_png, image)
    print(f"OpenCVを使用して寸法線のみのPNGを作成しました: {output_png}")

def main():
    #input_path = "/content/cubicasa5k/cubicasa5k/colorful/10106/F1_scaled.png"
    for i in range(1, 20230):
        input_path = f"svg/{i}.svg"


        output_path = f"png/modified_{i}.png"
        if True:
            save_svg_as_png_with_dimensions_only(input_path, output_path)
        else:
            extract_dimension_lines_to_png(input_path, output_path, "white")


if __name__ == "__main__":
    import sys
    sys.exit(main())

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import os
import argparse
from pathlib import Path
from glob import glob

def merge_floorplan_and_dimensions(floorplan_path, dimension_path, output_path):
    """
    フロアプラン画像と寸法線画像を合成する
    寸法線画像の白い部分は透過し、オブジェクト（黒い部分）のみを重ねる

    Args:
        floorplan_path: フロアプラン画像のパス
        dimension_path: 寸法線画像のパス
        output_path: 出力画像のパス
    """
    # 画像の読み込み
    floorplan = cv2.imread(floorplan_path)
    if floorplan is None:
        raise FileNotFoundError(f"フロアプラン画像が読み込めません: {floorplan_path}")

    dimensions = cv2.imread(dimension_path)
    if dimensions is None:
        raise FileNotFoundError(f"寸法線画像が読み込めません: {dimension_path}")

    # 画像のサイズを合わせる
    # フロアプランと同じサイズにリサイズ
    # 各画像の幅・高さを取得
    h_f, w_f = floorplan.shape[:2]
    h_m, w_m = dimensions.shape[:2]

    # 2つの画像のうち、小さい幅を基準にする
    target_width = min(w_f, w_m)

    # 小さい方を採用して左上基準でクロップ
    target_height = min(h_f, h_m)
    floorplan = floorplan[0:target_height, 0:target_width]
    dimensions_resized = dimensions[0:target_height, 0:target_width]

    # 寸法線画像からマスクを作成（非白部分を検出）
    # 白色は [255, 255, 255] なので、すべてのピクセルが255でない場所を検出
    mask = np.any(dimensions_resized < 250, axis=2).astype(np.uint8) * 255

    # マスクを3チャンネルに拡張
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # 合成: フロアプラン画像をベースに、マスク部分だけ寸法線画像を使う
    result = np.where(mask_3ch > 0, dimensions_resized, floorplan)

    # 結果を保存
    cv2.imwrite(output_path, result)
    print(f"合成画像を保存しました: {output_path}")

def main():

    for i in range(1, 5001):
        floorplan_path = f"/content/mask2former_resize/white_original_floorplan_{i}.png"
        dimension_path = f"png/modified_{i}.png"

        output_path = f"/content/mask2former_resize/modified_original_floorplan_{i}.png"

        merge_floorplan_and_dimensions(floorplan_path, dimension_path, output_path)

    return 0

if __name__ == "__main__":
    import sys
    import re
    sys.exit(main())

In [ ]:
import cv2
import os
from tqdm import tqdm

def align_resize_and_crop(input_dir, output_dir):
    """
    フロアプラン画像 (original_floorplan_{i}.png) と
    マスク画像 (room_mask_color_{i}.png) を以下の手順で処理する:
      1. 2つの画像の幅のうち、小さい方の幅に合わせて大きい方を縮小する。
         - 幅の比率を基準に各画像を縮小（小さい画像はそのまま）。
      2. 縮小後、両画像の高さが異なる場合は、左上を基準に
         高さが小さい方のサイズにクロップする。
    最終的に、両画像は同じサイズとなる。
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in tqdm(range(1, 6001)):
        floorplan_path = os.path.join(input_dir, f"modified_original_floorplan_{i}.png")
        mask_path      = os.path.join(input_dir, f"room_mask_color_{i}.png")

        if not (os.path.exists(floorplan_path) and os.path.exists(mask_path)):
            continue

        floorplan = cv2.imread(floorplan_path)
        mask_img  = cv2.imread(mask_path)

        if floorplan is None or mask_img is None:
            continue

        # 各画像の幅・高さを取得
        h_f, w_f = floorplan.shape[:2]
        h_m, w_m = mask_img.shape[:2]

        # 2つの画像のうち、小さい幅を基準にする
        target_width = min(w_f, w_m)

        # 小さい方を採用して左上基準でクロップ
        target_height = min(h_f, h_m)
        floorplan_cropped = floorplan[0:target_height, 0:target_width]
        mask_cropped      = mask_img[0:target_height, 0:target_width]

        # 出力ファイルパス
        out_floorplan = os.path.join(output_dir, f"original_floorplan_{i}.png")
        out_mask      = os.path.join(output_dir, f"room_mask_color_{i}.png")

        cv2.imwrite(out_floorplan, floorplan_cropped)
        cv2.imwrite(out_mask, mask_cropped)

if __name__ == "__main__":
    input_folder = "mask2former_resize"        # 元画像が置いてあるフォルダ
    output_folder = "mask2former_resize2" # 出力先フォルダ

    align_resize_and_crop(input_folder, output_folder)

In [ ]:
pip install cairosvg

In [ ]:
!rm svg/*.svg

In [ ]:
!rm png/*.png

In [ ]:
!rm -r mask2former

In [ ]:
!mkdir mask2former_resize2